In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from sklearn.linear_model import LinearRegression

In [11]:
df = pd.read_csv('CGMData/CGM_with_features.csv')
display(df)

,Readings (CGM / BGM),DateTime,Hour,Minute,Eating,Activity Level,Carbohydrate Intake,Insulin Dosage
0,153,2024-03-05 00:03:00,0,3,No,Low,0,2.0
1,161,2024-03-05 00:08:00,0,8,No,Low,0,2.0
2,171,2024-03-05 00:13:00,0,13,No,Low,0,2.0
3,185,2024-03-05 00:18:00,0,18,No,Low,0,2.0
4,198,2024-03-05 00:23:00,0,23,No,Low,0,2.0
...,...,...,...,...,...,...,...,...
3972,400,2024-03-18 23:36:00,23,36,No,Low,0,2.4
3973,400,2024-03-18 23:41:00,23,41,No,Low,0,2.4
3974,400,2024-03-18 23:46:00,23,46,No,Low,0,2.4
3975,400,2024-03-18 23:51:00,23,51,No,Low,0,2.4


In [16]:
start_date = '2024-03-17'
end_date = '2024-03-18'
recent_data = df[(df['DateTime'] >= start_date) & (df['DateTime'] <= end_date)]

# Assuming 'recent_data' is the filtered DataFrame with the past three days' data
X = recent_data[['Hour', 'Minute', 'Eating', 'Activity Level', 'Carbohydrate Intake', 'Insulin Dosage']]
X = pd.get_dummies(X, columns=['Eating', 'Activity Level'], drop_first=True)  # Convert categorical variables to dummy variables
y = recent_data['Readings (CGM / BGM)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize and train the Random Forest Regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Assuming 'hour' and 'minute' are columns in your DataFrame
next_hour = recent_data['Hour'].iloc[-1]  # Assuming the last entry in the DataFrame is the latest hour
next_minute = recent_data['Minute'].iloc[-1] + 30  # Adding 30 minutes to the latest minute
if next_minute >= 60:
    next_hour += 1
    next_minute -= 60
next_data_point = [[next_hour, next_minute, 0, 0, 0, 0]]  # Replace 0s with actual values for Eating, Activity Level, Carbohydrate Intake, and Insulin Dosage
next_prediction = rf_model.predict(next_data_point)
print(f'Predicted glucose reading for {next_hour:02d}:{next_minute:02d} and the next 30 minutes: {next_prediction[0]}')

Mean Squared Error: 450.4046379310344
Predicted glucose reading for 24:26 and the next 30 minutes: 167.81


c:\Users\zar10\miniconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
